# Matrix Factorization-based Collaborative Filtering

In this tutorial, you will see an example of using a strong Matrix Factorization (MF) method for recommendation, namely [SVD++](https://https://dl.acm.org/doi/pdf/10.1145/1401890.1401944). Please follow the instructions step-by-step as they try to explain different stages of developing an MF-based recommender.

**Step1:** Importing necessary packages and loading the data

First things first, we begin by install a group of necessary packages that are used in this notebook. The main ones are numpy and torch with which you probably are familiar. Major MF methods are already implemented in recommendation packages such as [Surprise](https://https://surpriselib.com/) and [LightFM](https://https://making.lyst.com/lightfm/docs/home.html), but here you can see a from scratch implementation. The main reasons for this are developing a deeper acquaintance for under-the-hood details of how an MF-based recommender works, and the fact that most of these excellent packages are not optimal for training using GPUs, but we need the acceleration caused by GPU and autodiff tools for gradient descent-based real-world applications on massive datasets.

In [2]:
import os, sys, re, pickle, torch
import numpy as np
from numpy.random import default_rng
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import time
import sys, os
import pickle
from torch import nn
from collections import defaultdict

In [3]:
##Importing the data file from google drive

from google.colab import drive
drive.mount('/content/drive/')          # this will direct you to a link where you can get anmhkk authorization key
import sys
sys.path.append('/content/drive/My Drive/')
##Changing the working directory

%cd '/content/drive/My Drive/'


Mounted at /content/drive/
/content/drive/My Drive


In [4]:
%cd VectorProject/

/content/drive/My Drive/VectorProject


In this folder, you are provided with three data files. "u.data" is the ratings data file of [MovieLens 100K](https://https://grouplens.org/datasets/movielens/100k/), "ML_ratings" includes a larger version of the same dataset, [MovieLens 20m ](https://grouplens.org/datasets/movielens/20m/), and LFM_ratings are the ratings from [LastFM ](https://https://grouplens.org/datasets/hetrec-2011/). You are more than welcome to explore working with the larger two datasets, but due to the greater training time, we just use MovieLens 100k for this showcase tutorial.

In [5]:
ls

LFM_ratings.txt  ML_ratings.txt  u.data


In [6]:

def u_i_dict_maker(rec):
  """
    Calculates the user to item mapping dictionary. Keys of this dict are 
    user ids and values are the list of movies each user likes
    """

  u_i_dict = defaultdict(list)
  for line in enumerate(rec):
    if line[1][1] not in u_i_dict[line[1][0]]:
      u_i_dict[line[1][0]].append(line[1][1])
  
  return u_i_dict


def load_rec_data(rec_path):
  """
    Reads the ratings data from the text files and converts them to numpy arrays.
    Next, it randomly splits the data to train, validation, and test sets.
    Finally, it forms the user to item mapping dicts for each split.
  """
  rec = np.genfromtxt(rec_path, delimiter='\t', dtype=np.int32)
  rec = rec[:,:3]
  rec_mapped = 1*rec
  unique_users = np.unique(rec[:,0])
  unique_items = np.unique(rec[:,1])
  users_map = {val: i for i, val in enumerate(unique_users)}
  items_map = {val: i for i, val in enumerate(unique_items)}

  ufunc = np.vectorize(lambda x: users_map[x])
  ifunc = np.vectorize(lambda x: items_map[x])

  # the mapped recommendation matrix where both user and item ids are 0,1,2,...
  np.random.seed(42)
  rec_mapped[:,0] = ufunc(rec[:,0])
  rec_mapped[:,1] = ifunc(rec[:,1])

  # split to train, test, and val
  train_ratio = 0.7 # 70% of data for training
  val_ratio = 0.15 # 15% of data for validation
  test_ratio = 0.15 # 15% of data for testing
  indices = np.arange(rec_mapped.shape[0])
  np.random.shuffle(indices)
  # split the data array into train, validation, and test sets
  train_indices = indices[:int(train_ratio*len(indices))]
  val_indices = indices[int(train_ratio*len(indices)):int((train_ratio+val_ratio)*len(indices))]
  test_indices = indices[int((train_ratio+val_ratio)*len(indices)):]

  train_data = rec_mapped[train_indices]
  val_data = rec_mapped[val_indices]
  test_data = rec_mapped[test_indices]
  # remove users from test and val that are not present in train
  #(we can't test on users for which we didn't have any training data. Although SVD++ is useful for the cold-start problem,
  # you often want to study that experiment separately)
  train_users = train_data[:,0]
  val_data = val_data[np.isin(val_data[:,0], train_users)]
  test_data = test_data[np.isin(test_data[:,0], train_users)]


  #maps each user to the list of items she interacted with

  u_i_dict_train, u_i_dict_val, u_i_dict_test = u_i_dict_maker(train_data), u_i_dict_maker(val_data), u_i_dict_maker(test_data)


  return rec_mapped, train_data, val_data, test_data, u_i_dict_train, u_i_dict_val, u_i_dict_test

In [7]:

ML_mapped, train_ML, val_ML, test_ML, u_i_dict_train_ML, u_i_dict_val_ML, u_i_dict_test_ML = load_rec_data('u.data')


Let's have a look at the data statistics. As you see, MovieLens 100k is a minuscule dataset usually used for tutorials rather than research.

In [8]:

print('number of users in MovieLens is: {}'.format(np.unique(ML_mapped[:,0]).shape[0]))
print('number of items in MovieLens is: {}'.format(np.unique(ML_mapped[:,1]).shape[0]))


number of users in MovieLens is: 943
number of items in MovieLens is: 1682


# Step 2. Define the model structure


<br>
SVD++ estimates the rank of an item i given by user u as:

- Model definition
$$
\hat{\mathbf{R}}_{ui} = \mu + b_u + b_i + \mathbf{q}_{i}^{T} \left( \mathbf{p}_{u} + |N(u)|^{-1/2} \sum_{j\in N(u)}\mathbf{y}_{j} \right) $$

In which:
  - $\hat{\mathbf{R}}_{ui}\in \mathbb{R}$: predicted rating user $u$ gives to item $i$
  - $\mu \in \mathbb{R}$: global bias
  - $b_{u} \in \mathbb{R}$: user bias
  - $b_{i} \in \mathbb{R}$: item bias
  - $\mathbf{q}_{i} \in \mathbb{R}^{1\text{x} k}$: $i^{\text{th}}$ row of $\mathbf{Q}$
  - $\mathbf{p}_{u} \in \mathbb{R}^{1\text{x} k}$: $u^{\text{th}}$ row of $\mathbf{P}$
  - $N(u) = \{i: | \, \mathbf{R}_{ui} \text{ is known} \}$: all items for which user $u$ provided a rating
  - $\mathbf{y}_{j} \in \mathbb{R}^{1\text{x}k}$: another item latent vector for implicit feedback


  - where, 
    - $k << m, n$: latent factor size
    - $\hat{\mathbf{R}} \in \mathbb{R}^{m \text{x} n}$: predicted rating matrix 
    - $\mathbf{P} \in \mathbb{R}^{m \text{x}k}$: user latent matrix
    - $\mathbf{Q} \in \mathbb{R}^{k \text{x}n}$: item latent matrix

<br>

- This estimation is found by minimizing the following Objective function:
$$
\underset{\mathbf{P}, \mathbf{Q}, \mathbf{y}_{*}, b}{\mathrm{argmin}} \sum_{(u, i) \in \mathcal{K}} \| \mathbf{R}_{ui} -
\hat{\mathbf{R}}_{ui} \|^2 + \lambda (\| \mathbf{P} \|^2_F + \| \mathbf{Q}
\|^2_F + b_u^2 + b_i^2 +\sum_{j\in N(u)} \|\mathbf{y}_{j}\|^2)
$$
Where:
  - $\lambda$: regularization rate (hyper-param)
  - $\mathcal{K}=\{(u, i) \mid \mathbf{R}_{ui} \text{ is known}\}$: The $(u,i)$ pairs for which $\mathbf{R}_{ui}$ is known and stored in the set


In [9]:
from torch import nn

class SVDpp(nn.Module):
    def __init__(self, num_factors, num_users, num_items, device, **kwargs):
        super(SVDpp, self).__init__(**kwargs)
        self.device = device
        # plain MF params
        self.P = nn.Embedding(num_users, num_factors).to(self.device)
        self.Q = nn.Embedding(num_items, num_factors).to(self.device)
        self.user_bias = nn.Embedding(num_users, 1).to(self.device)
        self.item_bias = nn.Embedding(num_items, 1).to(self.device)
        self.global_bias = nn.Parameter(torch.zeros(1)).to(self.device)
        # implicit feedback params
        self.y_j = nn.Embedding(num_items, num_factors).to(self.device) 

    def forward(self, user_id, item_id, u_i_dict):
        
        P_u = self.P(user_id)
        Q_i = self.Q(item_id)
        b_u = self.user_bias(user_id)
        b_i = self.item_bias(item_id)
        mu = self.global_bias

        # incoporating implicit feedback
        u_impl_list = [] 
        for u in user_id:
          u_i = u_i_dict[u.item()]
          u_impl = self.y_j(torch.tensor(u_i).to(self.device)).sum(axis=0)
          u_impl_list.append(u_impl / torch.tensor(len(u_i)).sqrt())
        u_impl_vec = torch.stack((u_impl_list))
        
        P_u += u_impl_vec

        if len(b_u) < 2:
          outputs = mu + torch.squeeze(b_u) + torch.squeeze(b_i) + (P_u * Q_i).sum() 
        else:
          outputs = mu + torch.squeeze(b_u) + torch.squeeze(b_i) + (P_u * Q_i).sum(axis=1) 
        return outputs.flatten()


After defining the model structure, for efficient computations using pyTorch, we form our data as Torch tensors and get them on the pyTorch DataLoader which helps us with efficiently shuffling and batching the data for the training and testing procedures.

In [10]:
def data_forming(train_ML, val_ML, test_ML, batch_size=256):
  """
  gets the dataset splits, converts it to torch tensors, and gets them on the DataLoader
  """
  train_u, train_i, train_r = train_ML[:,0], train_ML[:,1], train_ML[:,2]
  val_u, val_i, val_r = val_ML[:,0], val_ML[:,1], val_ML[:,2]
  test_u, test_i, test_r = test_ML[:,0], test_ML[:,1], test_ML[:,2]
  # Get on TensorDataset
  train_set = torch.utils.data.TensorDataset(
    torch.tensor(train_u), torch.tensor(train_i), torch.tensor(train_r, dtype=torch.float32))
  val_set = torch.utils.data.TensorDataset(
    torch.tensor(val_u), torch.tensor(val_i), torch.tensor(val_r))
  test_set = torch.utils.data.TensorDataset(
    torch.tensor(test_u), torch.tensor(test_i), torch.tensor(test_r))
  
  # Get on DataLoader
  train_iter = torch.utils.data.DataLoader(
      train_set, shuffle=True, batch_size=batch_size)
  val_iter = torch.utils.data.DataLoader(
      val_set, shuffle=True, batch_size=batch_size)
  test_iter = torch.utils.data.DataLoader(
      test_set, shuffle=True, batch_size=batch_size)
  
  return train_iter, val_iter, test_iter


Finally, we can train our model with the hyperparameter selection that you want. Generally, you want to use a hyperparameter tuning package such as [Ray](https://https://docs.ray.io/en/latest/tune/index.html) or [Weights and Biases](https://wandb.ai/site) for real-world problems, but since that is not the focal point of our interest here, we just manually define our hyperparameters. You are strongly encouraged to repeat training and testing with other hyperparameter settings to make conclusions about the influence of each hyperparameter. 

In [17]:
# model hyperparameters 
config={
    "num_factors":5,
    "batch_size":2048,
    'model_type': 'SVDpp',
    "optimizer": 'Adam',
    "wd":  1e-5 ,
    "lr": 0.01,
    "num_epochs":100,
    "save_every":100
}



# get the formed data splits
train_loader, val_loader, test_loader = data_forming(train_ML, val_ML, test_ML, config["batch_size"])


def train_eval_model(data, u_i_dict_train, u_i_dict_val, config, device):
  """
  Performs training and validation using the defined model and hyperparameters.
  """

  # we need these for model configuration
  num_users = np.unique(data[:,0]).shape[0]
  num_items = np.unique(data[:,1]).shape[0]
  

  # make an instance of the model. Here, the only model type we implemented is SVD++,
  # but using the same skeleton, you can easily define other MF-based models as well
  if config['model_type'] == 'SVDpp':
    model = SVDpp(config['num_factors'], num_users, num_items, device)

  # Define a loss function (MSE is the go-to choice for explicit rating data), but you
  # can define other metrics as well
  loss_fn = nn.MSELoss(reduction='mean')

  # Define an optimizer
  if config["optimizer"] == "Adam":
    optimizer = torch.optim.Adam((param for param in model.parameters()
                            if param.requires_grad), 
                         weight_decay=config["wd"], lr=config["lr"])
    
  else:
    optimizer = torch.optim.SGD((param for param in model.parameters()
                            if param.requires_grad), 
                         weight_decay=config["wd"], lr=config["lr"])
    

  ######################################################################
  # Train & Eval
  ######################################################################

  # Train
  for epoch in tqdm(range(config['num_epochs'])):
    tr_rmse = 0
    model.train()
    for u, i, r in train_loader:
      u, i, r = u.to(device), i.to(device), r.to(device)
      optimizer.zero_grad()
      output = model(u, i, u_i_dict_train)
      l = loss_fn(output, r)
      l.backward()
      optimizer.step()
      with torch.no_grad():
        tr_rmse += np.sqrt(loss_fn(output, r).cpu().numpy())
      
    print("\n training RMSE Loss: {}".format(tr_rmse))
    
    # Evaluate on Valid-set
    val_rmse = 0
    model.eval()
    for u, i, r in val_loader:
      u, i, r = u.to(device), i.to(device), r.to(device)
      r_hat = model(u, i, u_i_dict_val)
      with torch.no_grad():
        val_rmse += np.sqrt(loss_fn(r_hat, r).cpu().numpy())

    print("\t validation RMSE Loss: {}".format(val_rmse))

    if epoch % config["save_every"] + 1 == 0:
      #path = os.getcwd()
      torch.save((model.state_dict(), optimizer.state_dict()))


def try_gpu(i=0): 
    return f'cuda:{i}' if torch.cuda.device_count() >= i + 1 else 'cpu'

Check if you are connected to a GPU runtime

In [18]:
device = try_gpu()
print(device)



cuda:0


Finally run the training loop

In [19]:
train_eval_model(ML_mapped, u_i_dict_train_ML, u_i_dict_val_ML, config, device)

  0%|          | 0/100 [00:00<?, ?it/s]


 training RMSE Loss: 156.27580451965332


  1%|          | 1/100 [00:27<44:48, 27.16s/it]

	 validation RMSE Loss: 34.672574043273926

 training RMSE Loss: 124.96423649787903


  2%|▏         | 2/100 [00:54<44:24, 27.19s/it]

...

	 validation RMSE Loss: 15.839410662651062

 training RMSE Loss: 28.198251605033875


 99%|█████████▉| 99/100 [44:41<00:26, 26.58s/it]

	 validation RMSE Loss: 15.763977289199829

 training RMSE Loss: 28.21927011013031


100%|██████████| 100/100 [45:07<00:00, 27.08s/it]

	 validation RMSE Loss: 15.731103777885437


TODO: Now that you have selected your best model considering hte validation loss, please load the best saved model and test it on the test set

In [ ]:
##your code goes here: